In [1]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras._tf_keras.keras import layers, models
from keras._tf_keras.keras.optimizers import Adam
from keras._tf_keras.keras.losses import BinaryCrossentropy
import cv2

In [2]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Constants
VOLUME_SIZE = (128, 128, 128)  # Resize volumes to 128x128x128
BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-6

In [4]:
# Load 3D NIfTI file
def load_nifti(file_path):
    nifti = nib.load(file_path)
    data = nifti.get_fdata()
    return np.array(data, dtype=np.float32)

In [5]:
def preprocess_volume(volume, target_size):
    """
    Resize and normalize a 3D volume.
    """
    # Calculate zoom factors
    zoom_factors = (
        target_size[0] / volume.shape[0],
        target_size[1] / volume.shape[1],
        target_size[2] / volume.shape[2]
    )
    
    # Resize the volume using scipy.ndimage.zoom
    volume = zoom(volume, zoom_factors, order=1)  # order=1 for linear interpolation
    
    # Normalize to [0, 1]
    volume = volume / np.max(volume)
    
    # Add channel dimension
    volume = np.expand_dims(volume, axis=-1)
    
    return volume

In [6]:
# Load dataset
def load_dataset(image_dir, label_dir):
    images = []
    masks = []
    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        label_path = os.path.join(label_dir, img_file)
        
        # Load 3D NIfTI files
        img_volume = load_nifti(img_path)
        mask_volume = load_nifti(label_path)
        
        # Preprocess volumes
        img_volume = preprocess_volume(img_volume, VOLUME_SIZE)
        mask_volume = preprocess_volume(mask_volume, VOLUME_SIZE)
        
        images.append(img_volume)
        masks.append(mask_volume)
    
    return np.array(images), np.array(masks)



In [7]:
# Build and compile the model (3D U-Net)
def unet_3d(input_size=(128, 128, 128, 1)):
    inputs = layers.Input(input_size)
    
    # Encoder
    conv1 = layers.Conv3D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv3D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    conv2 = layers.Conv3D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv3D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    
    conv3 = layers.Conv3D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv3D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
    # Bottleneck
    conv4 = layers.Conv3D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv3D(256, 3, activation='relu', padding='same')(conv4)
    
    # Decoder
    up5 = layers.Conv3DTranspose(128, 2, strides=(2, 2, 2), padding='same')(conv4)
    up5 = layers.concatenate([up5, conv3])
    conv5 = layers.Conv3D(128, 3, activation='relu', padding='same')(up5)
    conv5 = layers.Conv3D(128, 3, activation='relu', padding='same')(conv5)
    
    up6 = layers.Conv3DTranspose(64, 2, strides=(2, 2, 2), padding='same')(conv5)
    up6 = layers.concatenate([up6, conv2])
    conv6 = layers.Conv3D(64, 3, activation='relu', padding='same')(up6)
    conv6 = layers.Conv3D(64, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv3DTranspose(32, 2, strides=(2, 2, 2), padding='same')(conv6)
    up7 = layers.concatenate([up7, conv1])
    conv7 = layers.Conv3D(32, 3, activation='relu', padding='same')(up7)
    conv7 = layers.Conv3D(32, 3, activation='relu', padding='same')(conv7)
    
    # Output layer
    outputs = layers.Conv3D(1, 1, activation='sigmoid')(conv7)
    
    model = models.Model(inputs, outputs)
    return model


In [11]:
# Load dataset
image_dir = "C:/PEC-26/JIPMER-INTERN/Data/liver/imagesTr"
label_dir = "C:/PEC-26/JIPMER-INTERN/Data/liver/labelTr"
images, masks = load_dataset(image_dir, label_dir)

FileNotFoundError: No such file or no access: 'C:/PEC-26/JIPMER-INTERN/Data/liver/labelTr/liver_0.nii'

In [9]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

In [10]:
# Build and compile the model
model = unet_3d()
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])

In [11]:
# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)

1/1 ━━━━━━━━━━━━━━━━━━━━ 50s 50s/step - accuracy: 0.0186 - loss: 0.7140 - val_accuracy: 0.0200 - val_loss: 0.7021


In [12]:
# Save the model
model.save("3d_unet_liver_tumor.h5")

In [13]:
# Generate 3D segmentation output for a new volume
def predict_3d_segmentation(model, input_volume):
    input_volume = np.expand_dims(input_volume, axis=0)  # Add batch dimension
    prediction = model.predict(input_volume)
    prediction = (prediction > 0.5).astype(np.uint8)  # Apply threshold
    return prediction[0]  # Remove batch dimension

In [14]:
# Example: Predict segmentation for a validation volume
sample_volume = X_val[0]
predicted_mask = predict_3d_segmentation(model, sample_volume)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [15]:
# Save the predicted 3D mask as a NIfTI file
def save_nifti(data, output_path):
    nifti = nib.Nifti1Image(data, affine=np.eye(4))
    nib.save(nifti, output_path)

save_nifti(predicted_mask, "predicted_3d_mask.nii.gz")

In [ ]:
'''
from tensorflow.keras.models import load_model

# Load the model
model = load_model("3d_unet_liver_tumor.h5")

# Use the model for predictions
predictions = model.predict(X_test)
'''